In [11]:
from imports.TactileBag import TactileBag

tbag = TactileBag('../data/bags/small3')

In [12]:
tbag.parsed_bag.values[:, -3:]

Exception: Bag not parsed yet. Call parse before loading.

In [13]:
import math
tbag.parse(possible_angles=[math.radians(9)], N_examples=5, theta=[0, 0, math.pi/6, math.pi/3, math.pi/2], N_iters=12)

parsing events:   0%|          | 0/18529 [00:00<?, ?msg/s]

parsing other:   0%|          | 0/55320 [00:00<?, ?msg/s]

In [3]:
from imports.ExtractContactCases import ExtractContactCases

ex = ExtractContactCases('../data/bags/small1/', '../data/small/contact_extraction10')

In [4]:
ex.extract()

extracting contact timestamps:   0%|          | 0/13167 [00:00<?, ?it/s]

extracting contact timestamps:   0%|          | 0/60 [00:00<?, ?it/s]

saving
